# Load Graphs

In [2]:
import os
import glob

path="/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data"

def load_graphs(path):
    path_list=[]
    for folder in os.listdir(path):
        os.chdir(path)
        path_list.append(glob.glob(os.path.join(path,folder,'*.{}'.format("edgelist"))))      
    return path_list



## Load Graph for t=1

In [48]:
import networkx as nx

paths=load_graphs(path)
count=len(paths[0])
number=len(paths)

def create_network_t1(paths:list,count:int,number:int):
    edgelist=paths[number-1][count-1]
    G=nx.read_edgelist(edgelist,create_using=nx.DiGraph)
    return G


OSError: [Errno 36] File name too long: '/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data/1/98_graph_data.edgelist%s/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data/1/39_graph_data.edgelist%s/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data/1/45_graph_data.edgelist%s/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data/1/51_graph_data.edgelist%s/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data/1/67_graph_data.edgelist%s/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data/1/31_graph_data.edgelist%s/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data/1/11_graph_data.edgelist%s/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data/1/9_graph_data.edgelist%s/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data/1/24_graph_data.edgelist%s/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data/1/76_graph_data.edgelist%s/home/nikl

In [49]:
G=create_network_t1(paths,count,number)
print(G)

Graph with 100 nodes and 1522 edges


## Load Graph for t > 1

In [46]:
def create_network_t(paths:list,t:int,run:int):
    G=None
    for i in range(t):
        edgelist=paths[run][i]
        g=nx.read_edgelist(edgelist,create_using=nx.DiGraph)
        if G is None:
            G=g
        else:
            G=nx.compose(G,g)
    return G

In [55]:
G=create_network_t(paths,4,3)
print(G)
in_degrees=dict(G.in_degree(weight="weight"))
in_degrees

DiGraph with 100 nodes and 6266 edges


{'0': 1.4413920914642664,
 '69': 2.78389921208891,
 '55': 0.9128981078675953,
 '7': 0.5851131029154735,
 '6': 2.6219964841904955,
 '57': -5.527009615683059,
 '94': 1.8406603280307021,
 '11': -1.2802191922493498,
 '14': -0.09189598425548984,
 '76': -1.28210181889249,
 '82': 0.7621487119765142,
 '47': 0.11117545183065719,
 '2': 0.805960089980247,
 '42': -2.4845643403243747,
 '90': -0.6213049372171883,
 '45': 0.8784049588775497,
 '71': 1.9332068294328613,
 '87': -0.16669631951023206,
 '91': -2.658761708603193,
 '98': 0.7864972767044245,
 '16': 0.6790964927551263,
 '51': -1.2785208573244131,
 '74': 0.285239997450679,
 '80': -0.39114745213865254,
 '70': 0.8411821536160731,
 '17': -1.2171560992501307,
 '40': -0.9277067295327587,
 '56': -0.18502556879120802,
 '77': 0.053192077120565825,
 '53': -0.9183626512821594,
 '48': 0.3413201022156972,
 '46': -1.9955579685130527,
 '3': 1.8136283946656242,
 '61': 1.4967996972938178,
 '84': -2.7384145142974186,
 '64': 0.5162290767976757,
 '24': 0.677163549

# Load model and agent data

In [18]:
def load_agent_data(path):
    path_list=[]
    for folder in os.listdir(path):
        os.chdir(path)
        path_list.append(glob.glob(os.path.join(path,folder,'agent_data.{}'.format("csv"))))      
    return path_list

agent_paths=load_agent_data(path)

In [51]:
import pandas as pd

def read_csv_t(paths:list,run:int):
    path="%s" % "','".join(paths[run])
    df=pd.read_csv(path)
    return df


In [52]:
df=read_csv_t(agent_paths,4)
df

,Step,AgentID,Info,Avg_PersonalizedTrust,GeneralizedTrust,Wealth,Suspectability,ID,SecurityLevel,Partner,Memory,Threshold
0,2,16,0.000000,-1.701401,-1.770539,0.571352,-0.362987,16,1.021078,50,8,0.702175
1,2,97,0.000000,-0.295401,-0.946604,-2.976599,1.161356,97,-3.490298,90,7,0.789599
2,2,98,0.000000,0.216092,0.840998,-2.088441,0.279939,98,2.416611,69,8,0.673516
3,2,14,0.000000,-0.669487,-2.254114,-3.100177,0.203161,14,0.537178,83,7,-0.059812
4,2,42,0.000000,-0.566904,-0.603277,0.225669,0.146266,42,5.775998,44,4,1.123787
...,...,...,...,...,...,...,...,...,...,...,...,...
9895,100,16,-0.213922,-0.702710,-1.770539,-154.225239,-0.362987,16,1.021078,0,8,0.702175
9896,100,50,0.000000,-0.479525,-1.288465,-160.549927,-0.124293,50,2.072625,32,5,0.533444
9897,100,22,0.000000,-0.162102,-0.708870,-159.672816,0.374204,22,3.689642,6,3,0.292406
9898,100,60,0.593755,-0.335553,-1.131045,-161.182540,0.376700,60,11.798907,71,8,0.706331


In [119]:
def create_degree_df(G:nx.DiGraph,step:int,out:bool):
    if out:
        degree_df = dict(G.out_degree(weight="weight"))
        degree_df = pd.DataFrame(degree_df, index=["out_degree"]).T
        degree_df = degree_df.reset_index()
        degree_df.rename(columns={"index":"ID"},inplace=True)
        degree_df["ID"]=degree_df["ID"].astype(int)
        degree_df["Step"]=step
    else:
        degree_df = dict(G.in_degree(weight="weight"))
        degree_df = pd.DataFrame(degree_df, index=["in_degree"]).T
        degree_df = degree_df.reset_index()
        degree_df.rename(columns={"index":"ID"},inplace=True)
        degree_df["ID"]=degree_df["ID"].astype(int)
        degree_df["Step"]=step
        #degree_df["Step"]=degree_df["Step"].astype(str)

    return degree_df

def merge_df(df1:pd.DataFrame,df2:pd.DataFrame):
    #df=pd.concat([df1,df2],axis=1)
    df=pd.merge(df1,df2,on=["Step","ID"])
    return df


In [126]:
#loop through data and construct final sets

def analyze_t(path:str):
    for i in range(20):
        agent_paths=load_agent_data(path)
        df=read_csv_t(agent_paths,i)
        paths=load_graphs(path)
        for j in range(100):
            g=create_network_t(paths,j,i)
            if g is not None:
                #calculate out-degrees
                out_degrees = create_degree_df(g,j,out=True)
                in_degrees = create_degree_df(g,j,out=False)
                df1=merge_df(df,out_degrees)
                df1=merge_df(df1,in_degrees)
                df1.to_csv(path+"/"+str(i+1)+"/"+str(j)+"step_data.csv")
                print("finished {}{}".format(i,j))
    return df
            
df=analyze_t("/home/niklas/Documents/Studium_Uni_Bamberg/Semester4/PWM-PT-HS5/data")
        


finished 01
finished 02
finished 03
finished 04
finished 05
finished 06
finished 07
finished 08
finished 09
finished 010
finished 011
finished 012


KeyboardInterrupt: 

# Sparsity/Clustering

- extract clustering per t per graph
- own df for network vars
- group by step
- create clustering histogram over t 


In [5]:
#calculate clustering over time show histogram
nx.average_clustering(G)


0.2972527536619025

# Centrality
- in-out degrees of nodes -> needed
- degree centrality (importance based in degrees)
- betweenness centrality (importance based on information flow)
- pagerank (importance metric for large and dense networks)

In [67]:
degrees = dict(nx.degree(G))
print(degrees)
#create pandas df to analyze degrees bzw add to pandas df (per step number)
degree_df = pd.DataFrame(degrees, index=["degre"]).T
degree_df = degree_df.reset_index()
degree_df["step"]=4
#degree_df.columns = ['degrees']
#degree_df.sort_values('degrees', inplace=True, ascending=False)
degree_df.head()

{'0': 128, '69': 116, '55': 126, '7': 126, '6': 112, '57': 114, '94': 122, '11': 140, '14': 120, '76': 124, '82': 120, '47': 132, '2': 120, '42': 132, '90': 118, '45': 126, '71': 132, '87': 132, '91': 132, '98': 128, '16': 122, '51': 116, '74': 124, '80': 122, '70': 116, '17': 130, '40': 126, '56': 116, '77': 120, '53': 126, '48': 126, '46': 132, '3': 118, '61': 136, '84': 124, '64': 118, '24': 130, '37': 128, '99': 120, '23': 128, '33': 138, '36': 124, '81': 126, '49': 118, '30': 122, '8': 114, '97': 128, '4': 122, '67': 126, '59': 130, '85': 120, '89': 128, '15': 128, '41': 120, '26': 126, '72': 122, '79': 124, '20': 122, '19': 120, '75': 128, '27': 126, '9': 120, '43': 134, '1': 126, '54': 132, '60': 126, '65': 118, '66': 134, '44': 132, '31': 128, '62': 126, '39': 130, '28': 138, '25': 120, '73': 130, '38': 128, '86': 124, '58': 132, '18': 116, '92': 130, '88': 122, '22': 136, '52': 126, '12': 122, '50': 118, '5': 122, '96': 128, '29': 124, '68': 118, '32': 130, '63': 132, '93': 12

,index,degre,step
0,0,128,4
1,69,116,4
2,55,126,4
3,7,126,4
4,6,112,4


In [ ]:
#out-degrees


In [ ]:
#degree centrality
degcent = nx.degree_centrality(G)

degcent_df = pd.DataFrame(degcent, index=[0]).T
degcent_df.columns = ['degree_centrality']
degcent_df.sort_values('degree_centrality', inplace=True, ascending=F
degcent_df.head()

In [ ]:
#betweenness centrality
betwcent = nx.betweenness_centrality(G)

betwcent_df = pd.DataFrame(betwcent, index=[0]).T
betwcent_df.columns = ['betweenness_centrality']
betwcent_df.sort_values('betweenness_centrality', inplace=True, ascen
betwcent_df.head()

In [ ]:
#pagerank
pagerank = nx.pagerank(G)

pagerank_df = pd.DataFrame(pagerank, index=[0]).T
pagerank_df.columns = ['pagerank']
pagerank_df.sort_values('pagerank', inplace=True, ascending=False)
pagerank_df.head()

In [ ]:
#combine
combined_importance_df = pd.concat([degree_df, degcent_df, betwcent_df])
combined_importance_df.head(10)

#sort by importance based on pagerank
combined_importance_df.sort_values('pagerank', ascending=False)

# Connected Components

In [ ]:
components = list(nx.connected_components(G))
len(components)